In [1]:
import os,sys,glob
from matplotlib import colors as mcolors
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
import seaborn as sns
from itertools import combinations
from sklearn.metrics import mean_squared_error
from scipy import stats

In [2]:
def checkForDuplicates(obj,finish_dates,subjects,sessions,paths):
    
    if (obj['output']['meta']['subject'] in subjects) and (obj['output']['meta']['session'] in sessions):
        index = np.where(np.logical_and(subjects == obj['output']['meta']['subject'],sessions == obj['output']['meta']['session']))
        if finish_dates[index] <= obj["finish_date"]:
            subjects = np.delete(subjects,index)
            paths = np.delete(paths,index)
            sessions = np.delete(sessions,index)
            finish_dates = np.delete(finish_dates,index)
    
    return finish_dates, subjects, sessions, paths

def addSubjectsSessions(subject,session,path,data):
    
    if 'subjectID' not in data.keys():
        data['subjectID'] = [ str(subject) for f in range(len(data)) ]
    
    if 'sessionID' not in data.keys():
        data['sessionID'] = [ str(session) for f in range(len(data)) ]
        
    return data

def appendData(subjects,sessions,paths,finish_dates,obj,filename):
        
    # check for duplicates. if so, remove
    finish_dates, subjects, sessions, paths = checkForDuplicates(obj,finish_dates,subjects,sessions,paths)

    subjects = np.append(subjects,obj['output']['meta']['subject'])
    sessions = np.append(sessions,obj['output']['meta']['session'])
    paths = np.append(paths,"input/"+obj["path"]+"/"+filename)
    finish_dates = np.append(finish_dates,obj['finish_date'])
    
    return finish_dates, subjects, sessions, paths

def compileData(paths,subjects,sessions,data):
    
    for i in range(len(paths)):
        if '.json.gz' in paths[i]:
            tmpdata = pd.read_json(paths[i],orient='index').reset_index(drop=True)
            tmpdata = addSubjectsSessions(subjects[i],sessions[i],paths[i],tmpdata)
        else:
            tmpdata = pd.read_csv(paths[i])
            tmpdata = addSubjectsSessions(subjects[i],sessions[i],paths[i],tmpdata)

        data = data.append(tmpdata,ignore_index=True)

    # replace empty spaces with nans
    data = data.replace(r'^\s+$', np.nan, regex=True)
    
    return data
    
def collectData(datatype,datatype_tags,tags,filename,outPath):

    import requests
    import pandas as pd

    # grab path and data objects
    objects = requests.get('https://brainlife.io/api/warehouse/secondary/list/%s'%os.environ['PROJECT_ID']).json()
    
    # subjects and paths
    subjects = []
    sessions = []
    paths = []
    finish_dates = []
    
    # set up output
    data = pd.DataFrame()

    for obj in objects:
        if obj['datatype']['name'] == datatype:
            if datatype_tags:
                if set(datatype_tags).issubset(obj['output']['datatype_tags']):
                    if tags:
                        if set(tags).issubset(obj['output']['tags']):
                            finish_dates, subjects, sessions, paths = appendData(subjects,sessions,paths,finish_dates,obj,filename)
                        elif '!' in str(tags):
                            tag = [ f for f in tags if '!' in str(f) ]
                            tag_drop = [ f for f in tags if f not in tag ]
                            if not set([ f.replace('!','') for f in tag]).issubset(obj['output']['tags']):
                                if set(tag_drop).issubset(obj['output']['tags']):
                                    finish_dates, subjects, sessions, paths = appendData(subjects,sessions,paths,finish_dates,obj,filename)
                    else:
                        finish_dates, subjects, sessions, paths = appendData(subjects,sessions,paths,finish_dates,obj,filename)
                elif '!' in str(datatype_tags):
                    datatype_tag = [ f for f in datatype_tags if '!' in str(f) ]
                    datatype_tag_drop = [ f for f in datatype_tags if f not in datatype_tag ]
                    if not set([ f.replace('!','') for f in datatype_tag]).issubset(obj['output']['datatype_tags']):
                        if tags:
                            if set(tags).issubset(obj['output']['tags']):
                                finish_dates, subjects, sessions, paths = appendData(subjects,sessions,paths,finish_dates,obj,filename)
                            elif '!' in str(tags):
                                tag = [ f for f in tags if '!' in str(f) ]
                                tag_drop = [ f for f in tags if f not in tag ]
                                if not set([ f.replace('!','') for f in tag]).issubset(obj['output']['tags']):
                                    if set(tag_drop).issubset(obj['output']['tags']):
                                        finish_dates, subjects, sessions, paths = appendData(subjects,sessions,paths,finish_dates,obj,filename)
                        else:
                            finish_dates, subjects, sessions, paths = appendData(subjects,sessions,paths,finish_dates,obj,filename)
            else:
                if tags:
                    if set(tags).issubset(obj['output']['tags']):
                        finish_dates, subjects, sessions, paths = appendData(subjects,sessions,paths,finish_dates,obj,filename)
                    elif '!' in str(tags):
                        tag = [ f for f in tags if '!' in str(f) ]
                        tag_drop = [ f for f in tags if f not in tag ]
                        if not set([ f.replace('!','') for f in tag]).issubset(obj['output']['tags']):
                            if set(tag_drop).issubset(obj['output']['tags']):
                                finish_dates, subjects, sessions, paths = appendData(subjects,sessions,paths,finish_dates,obj,filename)
                else:
                    finish_dates, subjects, sessions, paths = appendData(subjects,sessions,paths,finish_dates,obj,filename)

    # shuffle data
    paths = [z for _,_,z in sorted(zip(subjects,sessions,paths))]
    subjects = [x for x,_,_ in sorted(zip(subjects,sessions,paths))]
    sessions = [y for _,y,_ in sorted(zip(subjects,sessions,paths))]
    
    # compile data
    data = compileData(paths,subjects,sessions,data)

    # output data structure for records and any further analyses
    # subjects.csv
    data.to_csv(outPath,index=False)
    
    return data

In [4]:
# this will collect the data and save it to where you want
# collectData takes 5 input parameters:
# 1. datatype: this is the full datatype name for the datatype you want to grab
# 2. datatype_tags: this is a list of datatype tags corresponding to the appropriate datatype tags. this is useful for filtering only the datatype tags you'd like. you can set this
# as empty by setting this to ''
# 3. tags: this is a list of tags corresponding to the appropriate tags. this is useful for filtering only the tags you'd like. you can set this
# as empty by setting this to ''
# 4. filename: this is the filename of the specific file within the datatype.
# 5. outPath: this is the path to where you would like to save the data. example: './roi_diffusion.csv'


# here is an example for how to grab the parc-stats data with corresponding datatype_tags of 'rois' and 'diffusion' and with no tags. the file we're interested in is 'MEAN.csv'
# and we want to save the file to the current directory and name it 'rois_diffusion.csv'
roi_data = collectData('neuro/parc-stats',['rois','diffusion'],'','parc-stats/MEAN.csv','./rois_diffusion.csv')

In [5]:
roi_data

,subjectID,structureID,nodeID,ad,fa,md,rd,ga,ak,mk,rk,sessionID
0,S5,ROIfreesurfer-251,1,1.508937,0.700980,0.749453,0.369711,1.272534,NaN,NaN,NaN,SE1
1,S5,ROIfreesurfer-252,1,1.714621,0.605331,0.999165,0.641437,1.035202,NaN,NaN,NaN,SE1
2,S5,ROIfreesurfer-253,1,1.596885,0.468513,1.056818,0.786784,0.747447,NaN,NaN,NaN,SE1
3,S5,ROIfreesurfer-254,1,1.548117,0.466749,1.024044,0.762007,0.745595,NaN,NaN,NaN,SE1
4,S5,ROIfreesurfer-255,1,1.775762,0.572426,1.046419,0.681748,0.960580,NaN,NaN,NaN,SE1
5,S5,ROIfreesurfer-251,1,1.656669,0.722665,0.816258,0.396053,1.316635,NaN,NaN,NaN,SE10
6,S5,ROIfreesurfer-252,1,1.850152,0.592308,1.084015,0.700946,0.980323,NaN,NaN,NaN,SE10
7,S5,ROIfreesurfer-253,1,1.628007,0.539466,1.015059,0.708585,0.876036,NaN,NaN,NaN,SE10
8,S5,ROIfreesurfer-254,1,1.576563,0.497752,0.997264,0.707615,0.789727,NaN,NaN,NaN,SE10
9,S5,ROIfreesurfer-255,1,1.697728,0.575461,0.972454,0.609817,0.998577,NaN,NaN,NaN,SE10
